라이브러리 import

In [ ]:
import plotly.express as px

import pandas as pd

데이터 준비하기

In [ ]:
df = pd.read_csv('/content/도로교통공단_교통사고 정보.csv', encoding='euc-kr')
df.head(3)

In [ ]:
df.info()

교통사고 시간 전처리

In [ ]:
# substr 문자열 자르기를 위해 데이터타입 변경
df = df.astype({'발생년월일시' : 'string'})

In [ ]:
df.info()

In [ ]:
# 2019010100 -> 2019/01/01 00시 날짜(아직 문자열임)와 시간으로 분리 후 '발생시간'이라는 column 생성
df['발생시간'] = df['발생년월일시'].str[8:]

In [ ]:
df.info()

In [ ]:
# 발생시간 column의 데이터 타입을 문자형에서 숫자형으로 변환
df = df.astype({'발생시간' : 'int64'})

In [ ]:
df.info()

In [ ]:
# 2019010100 -> 20190101 추출 -> 2019-01-01 (날짜형태 YYYY-MM-DD 형태로 변경)
df['발생년월일시'] = pd.to_datetime(df['발생년월일시'].str[:8],
                              format='%Y-%m-%d', errors='raise') # format 지정할 때 Y는 대문자 m과 d는 소문자로 지정해야 한다

In [ ]:
df.info()

2019년 지역별 교통사고 데이터 시각화

In [ ]:
fig = px.scatter(df, x='발생년월일시', y='발생시간',
                 color='발생지시도',
                 hover_data=['발생시간']) # hover_data 는 scatter 차트에 커서를 댔을때 어떤 정보를 나타나게 할지 정하는 속성
fig.show()

In [ ]:
fig = px.scatter(df, x='발생년월일시', y='발생시간',
                 color='발생지시도',
                 size='사망자수',
                 hover_data=['발생시간']) # hover_data 는 scatter 차트에 커서를 댔을때 어떤 정보를 나타나게 할지 정하는 속성
fig.show()

2019년 시간대별 교통사고 사망자 현황

In [ ]:
fig = px.bar(df, x='사망자수', y='발생시간', orientation='h')
fig.show()

시도 별 교통사고 사망자 현황

In [ ]:
fig = px.bar(df, x='사망자수', y='발생지시도', orientation='h', title='시도별 교통사고 사망자 현황')
fig.show()

대구광역시의 교통사고 발생 데이터 분석

1. 대구광역시의 데이터 준비하기

In [ ]:
df1 = df[df['발생지시도'] == '대구']
df1.info()

2. 교통사고 발생시간 별 / 구 별 사망자 현황

In [ ]:
# 강조하고 싶은 부분을 x축에 놔야한다
fig = px.bar(df1, x='발생시간', y='사망자수', color='발생지시군구', title='대구광역시 교통사고 발생시간 별 / 구 별 사망자 현황')
fig.show()

2. -1. 발생시간 별 사망자 근황

In [ ]:
fig = px.bar(df1, x='사망자수', y='발생시간', color='발생지시군구', orientation='h', title='대구광역시 교통사고 발생시간 별 / 구 별 사망자 현황')
fig.show()

지도를 활용하여 교통하고 현황 분석

In [ ]:
import folium # 지도 라이브러리 import

map = folium.Map(location=[36.351,127.386]) # 목표로 하는 지역의 시청(의 좌표가) 중심이 된다

In [ ]:
map

지도에 필요한 정보만 담기 -> 대전의 교통사고 현황을 지도에 표츌

In [ ]:
df_map = df[df['발생지시도']=='대전']
df_map.info()

사망자수, 부상자수, 중상자수, 경상자수 전처리 -> 지도에 원을 그리기 위해서

In [ ]:
df_map = df_map.astype({'사망자수':'float64'})
df_map = df_map.astype({'부상자수':'float64'})
df_map = df_map.astype({'경상자수':'float64'})
df_map = df_map.astype({'중상자수':'float64'})

In [ ]:
df_map.info()

사망자 수, 경상자 수에 따른 CircleMarker 지도에 표출하기

In [ ]:
for n in df_map.index :
  cnt = df_map['사망자수'][n] + df_map['경상자수'][n] # 사망자수 + 경상자수 -> CircleMarker의 사이즈
  folium.CircleMarker([df_map['위도'][n], df_map['경도'][n]],
                      radius = cnt * 10, popup = df_map['사고유형'][n],
                      color='#3186cc', fill_color = '#3186cc'
                      ).add_to(map) # radius 원의 반지름 크기
map.save('대전교통사고 현황.html')

1) 선택한 지역의 교통사고 발생 시간 별/구 별 사망자 현황

In [ ]:
df2 = df[df['발생지시도']=='경기']

In [ ]:
fig2 = px.bar(df2, x='사망자수', y='발생지시군구', orientation='h', title='경기도 교통사고 시군구 별 사망자 현황')
fig2.show()

In [ ]:
my_df = df[df['발생지시군구']=='용인시']

In [ ]:
fig = px.bar(my_df, x='사망자수', y='발생시간', color='발생년월일시', orientation='h', title='용인시 교통사고 발생시간 별 사망자 현황')
fig.show()

In [ ]:
yongin_dead = my_df[(my_df['발생시간']==15)]
yongin_dead

In [ ]:
fig = px.bar(my_df, x='부상자수', y='발생시간', orientation='h', title='용인시 교통사고 발생시간 별 부상자 현황')
fig.show()

In [ ]:
fig = px.bar(my_df, x='경상자수', y='발생시간', orientation='h', title='용인시 교통사고 발생시간 별 경상자 현황')
fig.show()

In [ ]:
fig = px.bar(my_df, x='중상자수', y='발생시간', orientation='h', title='용인시 교통사고 발생시간 별 중상자 현황')
fig.show()

2) 지도 활용하여 교통사고 현황 분석


In [ ]:
# map에 용인시의 위치 정보를 저장
my_map = folium.Map(location=[37.241557269641646, 127.17806024603243])
my_map_dead = folium.Map(location=[37.241557269641646, 127.17806024603243])

# 숫자형을 실수형으로 변환
my_df = my_df.astype({'사망자수':'float64'})
my_df = my_df.astype({'부상자수':'float64'})
my_df = my_df.astype({'경상자수':'float64'})
my_df = my_df.astype({'중상자수':'float64'})

In [ ]:
my_df.index

In [ ]:
print(my_df['부상자수'])

In [ ]:
my_df['부상자수']

In [ ]:
for n in my_df.index :
  cnt = my_df['부상자수'][n]
  folium.CircleMarker([my_df['위도'][n], my_df['경도'][n]],
                      radius = cnt * 10, popup = my_df['사고유형'][n] + str(my_df['부상자수'][n]),
                      color='#3186cc', fill_color = '#3186cc'
                      ).add_to(my_map) # radius 원의 반지름 크기
my_map.save('용인시의 교통사고 부상자수 현황.html')

In [ ]:
for n in my_df.index :
  cnt = my_df['사망자수'][n]
  folium.CircleMarker([my_df['위도'][n], my_df['경도'][n]],
                      radius = cnt * 10, popup = my_df['사고유형'][n] + str(my_df['사망자수'][n]),
                      color='#3186cc', fill_color = '#3186cc'
                      ).add_to(my_map_dead) # radius 원의 반지름 크기
my_map_dead.save('용인시의 교통사고 사망자수 현황.html')